<a href="https://colab.research.google.com/github/Annisha-de-Souza/AES_Implementation/blob/main/ELDE_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision torchaudio
!pip install torch-kitti
!pip install kaggle --quiet

In [2]:
!pip install kaggle --quiet

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"annishadesouza","key":"69701b0f533bcc4a2d7c38c4dc2aa60b"}'}

In [4]:
mkdir ~/.kaggle

In [5]:
!cp kaggle.json ~/.kaggle/

In [10]:
ls -ltr ~/.kaggle

total 4
-rw------- 1 root root 70 Oct 23 04:20 kaggle.json


In [7]:
!chmod 600 ~/.kaggle/kaggle.json

In [9]:
!kaggle datasets list

ref                                                           title                                                    size  lastUpdated                 downloadCount  voteCount  usabilityRating  
------------------------------------------------------------  -------------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
ahmeduzaki/global-earthquake-tsunami-risk-assessment-dataset  Global Earthquake-Tsunami Risk Assessment Dataset       16151  2025-10-01 16:35:53.273000           9331        341  1.0              
jockeroika/life-style-data                                    Life Style Data                                       3995645  2025-10-14 13:50:45.303000          11185        244  0.8235294        
jaderz/hospital-beds-management                               Hospital Beds Management                                47583  2025-10-03 09:21:58.590000           7871        200  1.0              
ahmadrazakashif

In [12]:
!kaggle datasets download -d soumikrakshit/nyu-depth-v2


Dataset URL: https://www.kaggle.com/datasets/soumikrakshit/nyu-depth-v2
License(s): unknown
100% 4.10G/4.10G [00:48<00:00, 209MB/s]
100% 4.10G/4.10G [00:48<00:00, 91.8MB/s]


In [14]:
!ls



kaggle.json  nyu-depth-v2.zip  sample_data


In [15]:
!unzip "nyu-depth-v2.zip" -d nyu_depth_v2


Streaming output truncated to the last 5000 lines.
  inflating: nyu_depth_v2/nyu_data/data/nyu2_train/playroom_0004_out/32.jpg  
  inflating: nyu_depth_v2/nyu_data/data/nyu2_train/playroom_0004_out/32.png  
  inflating: nyu_depth_v2/nyu_data/data/nyu2_train/playroom_0004_out/33.jpg  
  inflating: nyu_depth_v2/nyu_data/data/nyu2_train/playroom_0004_out/33.png  
  inflating: nyu_depth_v2/nyu_data/data/nyu2_train/playroom_0004_out/34.jpg  
  inflating: nyu_depth_v2/nyu_data/data/nyu2_train/playroom_0004_out/34.png  
  inflating: nyu_depth_v2/nyu_data/data/nyu2_train/playroom_0004_out/35.jpg  
  inflating: nyu_depth_v2/nyu_data/data/nyu2_train/playroom_0004_out/35.png  
  inflating: nyu_depth_v2/nyu_data/data/nyu2_train/playroom_0004_out/36.jpg  
  inflating: nyu_depth_v2/nyu_data/data/nyu2_train/playroom_0004_out/36.png  
  inflating: nyu_depth_v2/nyu_data/data/nyu2_train/playroom_0004_out/37.jpg  
  inflating: nyu_depth_v2/nyu_data/data/nyu2_train/playroom_0004_out/37.png  
  inflating: 

In [16]:
!ls nyu_depth_v2


nyu_data


In [21]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [22]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list | head


ref                                                           title                                                    size  lastUpdated                 downloadCount  voteCount  usabilityRating  
------------------------------------------------------------  -------------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
ahmeduzaki/global-earthquake-tsunami-risk-assessment-dataset  Global Earthquake-Tsunami Risk Assessment Dataset       16151  2025-10-01 16:35:53.273000           9435        342  1.0              
jockeroika/life-style-data                                    Life Style Data                                       3995645  2025-10-14 13:50:45.303000          11221        245  0.8235294        
jaderz/hospital-beds-management                               Hospital Beds Management                                47583  2025-10-03 09:21:58.590000           7949        201  1.0              
ahmadrazakashif

In [2]:
import os
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset, ConcatDataset
from torch_kitti.transforms import ApplyToFeatures
from torchvision.transforms import Compose, ToTensor, RandomCrop

In [3]:
class NYUDepthV2Dataset(Dataset):
    def __init__(self, root_dir, csv_file, transform=None):
        self.root_dir = root_dir
        self.annotations = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        rgb_path = os.path.join(self.root_dir, self.annotations.iloc[idx]['rgb_path'])
        depth_path = os.path.join(self.root_dir, self.annotations.iloc[idx]['depth_path'])

        rgb_img = Image.open(rgb_path).convert("RGB")
        depth_img = Image.open(depth_path).convert("L")

        if self.transform:
            seed = torch.seed()
            torch.manual_seed(seed)
            rgb_img = self.transform(rgb_img)
            torch.manual_seed(seed)
            depth_img = self.transform(depth_img)

        return {'img': rgb_img, 'gt': depth_img}


In [4]:
class KITTIDepthDataset(Dataset):
    def __init__(self, root_dir, split='training', transform=None):
        self.root_dir = root_dir
        self.split = split
        self.transform = transform

        self.img_dir = os.path.join(root_dir, 'data_object_image_2', split)
        self.calib_dir = os.path.join(root_dir, 'data_object_calib', split)
        self.label_dir = os.path.join(root_dir, 'data_object_label_2', split) if split == 'training' else None

        self.image_files = sorted(os.listdir(self.img_dir))

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.image_files[idx])
        img = Image.open(img_path).convert("RGB")

        # For depth supervision, you need to load or generate depth map separately.
        # This example does not include depth loading. Insert your depth loading here.
        depth_gt = None

        if self.transform:
            img = self.transform(img)
            if depth_gt is not None:
                depth_gt = self.transform(depth_gt)

        sample = {'img': img}
        if depth_gt is not None:
            sample['gt'] = depth_gt

        return sample


In [5]:
class DepthDecoder(nn.Module):
    def __init__(self, in_channels, out_channels=1):
        super(DepthDecoder, self).__init__()
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(in_channels, 256, 3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(256, 128, 3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, out_channels, 3, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.decoder(x)


In [6]:
class DepthDecoder(nn.Module):
    def __init__(self, in_channels, out_channels=1):
        super(DepthDecoder, self).__init__()
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(in_channels, 256, 3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(256, 128, 3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, out_channels, 3, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.decoder(x)


In [7]:
class DepthDecoder(nn.Module):
    def __init__(self, in_channels, out_channels=1):
        super(DepthDecoder, self).__init__()
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(in_channels, 256, 3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(256, 128, 3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, out_channels, 3, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.decoder(x)


In [8]:
transform = Compose([
    RandomCrop([256, 512]),
    ToTensor()
])

# Please upload your KITTI and NYU datasets to your Google Drive and mount it.
# Replace the paths below with the correct paths in your Google Drive.
# You can mount your Google Drive by running: from google.colab import drive; drive.mount('/content/drive')
# Then you can access your files at '/content/drive/My Drive/your_dataset_folder'

kitti_dataset = KITTIDepthDataset(
    root_dir='/content/drive/My Drive/your_kitti_dataset_folder',  # Replace with your KITTI root folder in Google Drive
    split='training',
    transform=transform
)

nyu_dataset = NYUDepthV2Dataset(
    root_dir='/content/drive/My Drive/your_nyu_dataset_folder/nyu2_train', # Replace with your NYU dataset root folder in Google Drive
    csv_file='/content/drive/My Drive/your_nyu_dataset_folder/nyu2_train.csv', # Replace with the correct path to your CSV file in Google Drive
    transform=transform
)

combined_dataset = ConcatDataset([kitti_dataset, nyu_dataset])

data_loader = DataLoader(combined_dataset, batch_size=4, shuffle=True, num_workers=4)

# Ensure ELDE_Net is defined or imported before this cell
# model = ELDE_Net(pretrained=True)
# output_depth = model(input_img) # This line will be executed after defining the model.

print(f"Combined dataset size: {len(combined_dataset)}")
# print(f"Input image shape: {input_img.shape}") # These lines will be executed after getting a batch from the data_loader.
# print(f"Output depth map shape: {output_depth.shape}") # These lines will be executed after defining and using the model.

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\rache\\Uni Projects\\Disability-7 SEM\\dataset\\archive(1)/data_object_image_2/training'

In [ ]:
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(10):
    for batch in data_loader:
        input_img = batch['img']
        depth_gt = batch.get('gt')  # May be None if depth GT not available for KITTI

        optimizer.zero_grad()
        pred_depth = model(input_img)

        if depth_gt is not None:
            loss = criterion(pred_depth, depth_gt)
            loss.backward()
            optimizer.step()

            print(f"Epoch {epoch + 1}, Loss = {loss.item():.4f}")
        else:
            print("Batch without depth GT, skipping loss computation")


In [ ]:
import numpy as np

def abs_rel(pred, gt):
    return np.mean(np.abs(gt - pred) / gt)

def rmse(pred, gt):
    return np.sqrt(np.mean((gt - pred) ** 2))

def silog(pred, gt):
    d = np.log(pred) - np.log(gt)
    return np.sqrt(np.mean(d ** 2) - (np.mean(d) ** 2)) * 100

def threshold_acc(pred, gt, thr):
    return np.mean((np.maximum(pred / gt, gt / pred)) < thr)

def compute_metrics(pred, gt):
    pred = pred.cpu().numpy()
    gt = gt.cpu().numpy()

    mask = (gt > 0)
    pred = pred[mask]
    gt = gt[mask]

    absRel = abs_rel(pred, gt)
    rmseVal = rmse(pred, gt)
    silogVal = silog(pred, gt)
    d1 = threshold_acc(pred, gt, 1.25)
    d2 = threshold_acc(pred, gt, 1.25 ** 2)
    d3 = threshold_acc(pred, gt, 1.25 ** 3)

    return {
        'AbsRel': absRel, 'RMSE': rmseVal, 'SILog': silogVal,
        'Delta1.25': d1, 'Delta1.25^2': d2, 'Delta1.25^3': d3
    }


In [ ]:
model.eval()
metrics_sum = {
    'AbsRel': 0, 'RMSE': 0, 'SILog': 0,
    'Delta1.25': 0, 'Delta1.25^2': 0, 'Delta1.25^3': 0
}
num_samples = 0

with torch.no_grad():
    for batch in data_loader:
        input_img = batch['img']
        depth_gt = batch.get('gt')  # Ground truth depth maps

        if depth_gt is None:
            print("No ground truth depth in batch; skipping")
            continue

        pred_depth = model(input_img)
        batch_metrics = compute_metrics(pred_depth, depth_gt)

        for key in metrics_sum.keys():
            metrics_sum[key] += batch_metrics[key]
        num_samples += 1

# Average metrics across dataset
avg_metrics = {key: val / num_samples for key, val in metrics_sum.items()}
print("Test set evaluation metrics:")
for key, val in avg_metrics.items():
    print(f"{key}: {val:.4f}")
